In [1]:
import loader
import importlib
import pandas as pd
import utility
import spacy

importlib.reload(loader)


<module 'loader' from 'C:\\Users\\Roxanne El Baff\\Documents\\projects\\acl2020-news-editorial-analysis\\loader.py'>

In [ ]:
import glob
def clean_content_ending(article_content, id_):
    clean_end_content = end_pars[end_pars['id'] == int(id_.replace(".txt", ""))]['content'].values[0]
    idx_ = article_content.find(clean_end_content) +len(clean_end_content)
    if  article_content.find(clean_end_content)>-1:
        print('OK')
    else:
        print('ACHTUNG1 ',id_)
    #print('removed: ', row['content'][idx_:])
    removed= article_content[idx_:]
    content = article_content[:idx_]
    return content, removed

end_pars = load.pars_features_df[load.pars_features_df['discourse_level'] == 'end'][['id', 'content']]
corpus_txts_paths = glob.glob('corpus/*.txt')
removed_txts = []
for path in corpus_txts_paths:
    new_path = path.replace('corpus', 'cleaned_corpus')
    id_ = path.split('\\')[-1]

    with open(path, 'r', encoding="utf-8") as f:
        text = f.read().strip()
        text , removed= clean_content_ending(text, id_)
        removed_txts.append(removed)
        with open(new_path, 'w', encoding="utf-8") as w:
            w.write(text)
    
    


In [ ]:
removed_txts

In [2]:
load = loader.loader()
corpus = load.corpus
corpus = load.add_ideology_intensity()
#articles_per_ideology_intensity = load.get_article_dfs_per_ideology(ideology='ideology_intensity', include_content=True)
#articles_per_ideology_orientation =  load.get_article_dfs_per_ideology(ideology='political_pole', include_content=True)

"corpus" is set. It contains the 6000 annotation
"corpus" is set with ideology_intensity: "extreme" and "lean"


## Get Liberal and conservative df

In [ ]:

liberal_articles_df = articles_per_ideology_orientation['liberal']
conservative_articles_df = articles_per_ideology_orientation['conservative']

#liberal_articles_df=liberal_articles_df.apply(apply_add_majority, axis=1)
#conservative_articles_df=conservative_articles_df.apply(apply_add_majority, axis=1)

In [ ]:
liberal_articles_df['majority'].value_counts()

In [ ]:
conservative_articles_df['majority'].value_counts()

In [ ]:

def print_stats(df):
    for split, split_df in df.groupby('majority'):
        print(split)
        print(split_df['split_label'].value_counts()) 

In [ ]:
print_stats(conservative_articles_df)
print_stats(liberal_articles_df)

## Get ADUs

In [ ]:
discourse_adus_df = load.load_discourse_adus()
liberal_articles_df=liberal_articles_df.join(discourse_adus_df )
conservative_articles_df=conservative_articles_df.join(discourse_adus_df )
conservative_articles_df = conservative_articles_df[['majority']]
liberal_articles_df = liberal_articles_df[['ids', 'split_label',
       'majority', 'content', 'adu_anecdote', 'adu_other',
       'adu_statistics', 'adu_testimony']]
liberal_articles_df.rename(columns={"majority":"liberal_majority"}, inplace=True)
conservative_articles_df.rename(columns={"majority":"conservative_majority"}, inplace=True)
df = liberal_articles_df.join(conservative_articles_df)



In [ ]:
def get_train_test(df):
    return df[df['split_label'] == 'train'], df[df['split_label'] == 'test']



## Extract features - lexicon based

In [ ]:
import importlib
import lexicons


In [ ]:
df = lexicons.count_nrc_emotions_and_sentiments(df, text_column= 'content')
df = lexicons.count_mpqa_arg(df, text_column= 'content')
df = lexicons.count_mpqa_subj_obj(df)

## BERT 

In [ ]:
from collections import Counter
from simpletransformers.classification import ClassificationModel
import pandas as pd

def train_eval(train_df, eval_df):
    # Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns. If the Dataframe has a header, it should contain a 'text' and a 'labels' column. If no header is present, the Dataframe should contain at least two columns, with the first column is the text with type str, and the second column in the label with type int.
    #train_df = pd.DataFrame(train_data)

    #eval_data = [['Example eval sentence belonging to class 1', 1], ['Example eval sentence belonging to class 0', 0], ['Example eval senntence belonging to class 2', 2]]
    #eval_df = pd.DataFrame(eval_data)

    # Create a ClassificationModel
    model = ClassificationModel('bert', 'bert-base-uncased', num_labels=3, use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True})

    # Train the model


    model.train_model(train_df)

    # Evaluate the model
    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    #predictions, raw_outputs = model.predict(["Some arbitary sentence"])
    return model, result, model_outputs, wrong_predictions

def eval(wrong_predictions, model_outputs):
    len(wrong_predictions)#from collections import Counter
    #predictions, raw_outputs = model.predict(["Some arbitary sentence"])
    #model_outputs[0].argmax()

    labels = [x.argmax() for x in model_outputs]
    print(Counter(labels))
lib_model, lib_result, lib_model_outputs, lib_wrong_predictions = train_eval(train_liberal_df[['content', 'majority_int']], test_liberal_df[['content', 'majority_int']])
eval(lib_wrong_predictions, lib_model_outputs)
cons_model, cons_result, cons_model_outputs, cons_wrong_predictions  = train_eval(train_cons_df[['content', 'majority_int']], test_cons_df
           [['content', 'majority_int']])
eval(cons_wrong_predictions, cons_model_outputs)

